# TensorFlow Autoencodeur avec attention pour le PAr

## Setup

Moi j'ai installé tf addons par `pip install tensorflow-addons==0.13.0` (ET NON PAS `conda install -c esri tensorflow-addons`). Voir les compatibilités [sur le github de tensorflow_addons](https://github.com/tensorflow/addons).

In [1]:
# !pip install tensorflow-addons==0.11.2

In [1]:
import sys
print(sys.version)

3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:36:06) [MSC v.1929 64 bit (AMD64)]


In [2]:
import tensorflow as tf
import numpy as np
import io

print(tf.__version__)

2.7.0


In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

Num GPUs Available:  1


# Step 1: Get the data

In [4]:
path_reglement_scol  = './word2vec_docs_scol_traités/corpus.txt'
path_questions_scol  = './word2vec_docs_scol_traités/toutes-les-questions.txt'

# Step 2: Preprocess the data

In [5]:
import re as regex
# acquisition du texte
reglement_scol = io.open(path_reglement_scol, encoding='UTF-8').read()#.strip().split('\n')
questions_scol = io.open(path_questions_scol, encoding='UTF-8').read()#.strip().split('\n')
texte = reglement_scol + ' ' + questions_scol
texte = regex.sub("\n", " ", texte)

On crée d'abord une liste de phrases dont chaque mot est séparé par un espace. On a besoin de `spacy` pour découper correctement les mots en français d'abord.

In [6]:
import nltk
import spacy
nlp = spacy.load('fr_core_news_sm')
phrases = nltk.tokenize.sent_tokenize(texte, language='french')
print('phrases parsées par NLTK')
phrasesTokeniseesSpacy = [nlp(s) for s in phrases]
print('phrases tokénisées par spacy')
phrasesSpacy = [' '.join([token.text.lower() for token in doc]) for doc in phrasesTokeniseesSpacy]
print('phrases découpées en tokens puis refusionnées')

phrases parsées par NLTK
phrases tokénisées par spacy
phrases découpées en tokens puis refusionnées


On supprime les listes inutiles désormais

In [7]:
del phrasesTokeniseesSpacy
del phrases

Créer un tokéniseur adapté à notre vocabulaire

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters='')
# créer un tokenizer adapté à tout le vocabulaire des phrases
tokenizer.fit_on_texts(phrasesSpacy)

Créer les tenseurs pour toutes les phrases et padder le tout

In [9]:
tensor_sentences = tokenizer.texts_to_sequences(phrasesSpacy)
print(type(tensor_sentences))
print(phrasesSpacy[0],tensor_sentences[0])
# enfin on padd le tout pour pouvoir l'utiliser dans un réseau de neurones
tensor_sentences = tf.keras.preprocessing.sequence.pad_sequences(tensor_sentences,padding='post')

<class 'list'>
le règlement de scolarité présente les modalités d' admission à l' école centrale de lyon , les objectifs et les modalités de l' évaluation des connaissances et des compétences de la formation ingénieur , les modalités de diversification de cette formation et les conditions d' obtention des diplômes de l' école centrale de lyon , hors diplômes de master co-accrédités et diplôme d' ingénieur energie en alternance . [8, 131, 1, 59, 860, 4, 102, 6, 175, 9, 3, 25, 46, 1, 43, 10, 4, 861, 16, 4, 102, 1, 3, 77, 12, 90, 16, 12, 104, 1, 5, 42, 88, 10, 4, 102, 1, 1565, 1, 166, 42, 16, 4, 285, 6, 214, 12, 85, 1, 3, 25, 46, 1, 43, 10, 390, 85, 1, 247, 1189, 16, 22, 6, 88, 1190, 18, 615, 13]


In [10]:
# Fonction qui convertit un mot en son représentant entier
def convert(tokenizer, tensor):
    for t in tensor: # t est un entier élément du tenseur
        if t != 0:
            print ("%d ----> %s" % (t, tokenizer.index_word[t]))
convert(tokenizer, tensor_sentences[-1])

37 ----> comment
5 ----> la
38 ----> mobilité
211 ----> est-elle
1564 ----> vérifiée
15 ----> pour
4 ----> les
97 ----> doubles
85 ----> diplômes
18 ----> en
80 ----> france
2 ----> ?


# Step 3: Prepare the DataSet

`tokenizer.index_word` est un dictionnaire dont les clés sont des entiers et les valeurs sont des struings (mots du vocabulaire)

In [11]:
print('tensor:')
print(type(tensor_sentences))
print(np.shape(tensor_sentences))
tensor_sentences[0]
print("tokenizer:")
print(type(tokenizer))
print(type(tokenizer.index_word))

vocab_inp_size = len(tokenizer.word_index)
n_data,max_length = tensor_sentences.shape
embedding_dim = 16

print(f"nombre de données: {n_data}\nlongueur max phrases en mots: {max_length}\ntaille du vocabulaire: {vocab_inp_size}\ndimension de l'embedding: {embedding_dim}")

tensor:
<class 'numpy.ndarray'>
(2201, 347)
tokenizer:
<class 'keras_preprocessing.text.Tokenizer'>
<class 'dict'>
nombre de données: 2201
longueur max phrases en mots: 347
taille du vocabulaire: 2555
dimension de l'embedding: 16


# Step 4: Split the train and validation data

In [12]:
from sklearn.model_selection import train_test_split

# Create training and validation sets using an 80/20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(tensor_sentences, tensor_sentences, test_size=0.2)

print(type(input_tensor_train), type(target_tensor_train))
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

# on observe ce qu'il y a dans ces données: si on rééxécute ça change, c'est parce qu'il y a un shuffle aléatoire
convert(tokenizer, input_tensor_train[0])

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
1760 1760 441 441
28 ----> que
32 ----> peut
553 ----> offrir
7 ----> un
503 ----> congé
204 ----> sabbatique
2 ----> ?


# Step 5: create Encoder and Decoder classes

In [13]:
# Encoder class
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        self.enc_units = enc_units
        
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x):
        x = self.embedding(x)
        output, state = self.gru(x)
        return output, state
        # hidden state shape == (batch_size, hidden size)
        # output       shape == (batch_size, max_len, hidden size)

In [61]:
# Decoder class
class Decoder(tf.keras.Model):
    def __init__(self, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.attention = tf.keras.layers.Attention()


    def call(self, enc_output):
        attention_outputs, attention_scores = tf.keras.layers.Attention()([enc_output, enc_hidden], return_attention_scores=True)
        context = attention_outputs * enc_output
        return context,attention_scores

Call the encoder class to check the shapes of the encoder output and hidden state.

In [15]:
encoder = Encoder(vocab_inp_size, embedding_dim, 128)

enc_in = tf.random.uniform(
    (6,10),
    minval=0,
    maxval=60,
    dtype=tf.dtypes.int32,
    name="dummy_input_encoder"
)


print('Encoder Input        shape: (batch_size, timesteps)                {}'.format(enc_in.shape))
enc_output, enc_hidden = encoder(enc_in)

print('Encoder Output       shape: (batch_size, sequence_length, units)   {}'.format(enc_output.shape))
print('Encoder Hidden_state shape: (batch_size, units)                    {}'.format(enc_hidden.shape))

Encoder Input        shape: (batch_size, timesteps)                (6, 10)


AttributeError: Exception encountered when calling layer "encoder_1" (type Encoder).

'Encoder' object has no attribute 'embedding'

Call arguments received:
  • x=tf.Tensor(shape=(6, 10), dtype=int32)

In [59]:
class Autoencoder(tf.keras.Model):
    def __init__(self, dimension_embedding, vocab_inp_size, latent_dim):
        super().__init__()

        self.latent_dim = 128
        self.encoder = Encoder(vocab_inp_size, embedding_dim, latent_dim)
        self.decoder = Decoder(latent_dim)

    def call(self, inputs):
        enc_output,_ = self.encoder(inputs)
        out_dec = self.decoder(enc_output)

        return out_dec


In [60]:
latent_dim = 128
autoenc = Autoencoder(16,vocab_inp_size,128)
autoenc.compile(optimizer='Adam', loss=tf.losses.MeanSquaredError(), metrics = ["accuracy"]) # losses.MeanSquaredError() losses.CosineSimilarity()
autoenc.build(input_shape=input_tensor_train.shape)


enc_in.shape, autoenc(enc_in).shape

(TensorShape([6, 10]), TensorShape([6, 10, 128]))

In [48]:
latent_dim = 128
encoder = Encoder(vocab_inp_size, embedding_dim, latent_dim)
decoder = Decoder(latent_dim)

input = tf.keras.Input(shape=(max_length,embedding_dim))
out_enc = encoder(input)
out_dec = decoder(out_enc)
func_model = tf.keras.Model(inputs=input, outputs=out_dec)

ValueError: Exception encountered when calling layer "encoder_15" (type Encoder).

in user code:

    File "C:\Users\matth\AppData\Local\Temp/ipykernel_13788/546019233.py", line 18, in call  *
        output, state = self.gru(x)
    File "C:\Users\matth\.conda\envs\tf\lib\site-packages\keras\layers\recurrent.py", line 679, in __call__  **
        return super(RNN, self).__call__(inputs, **kwargs)
    File "C:\Users\matth\.conda\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\matth\.conda\envs\tf\lib\site-packages\keras\engine\input_spec.py", line 213, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}" '

    ValueError: Input 0 of layer "gru_15" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 347, 16, 16)


Call arguments received:
  • x=tf.Tensor(shape=(None, 347, 16), dtype=float32)